In [55]:
import numpy as np
import matplotlib as plt
from PIL import Image
from SHA_function import create_sha_key
from logistic_map import calculate_r_and_x
from generate_weights import create_weights
from forward_pass import Substitute , Perturbation
from Deferentail_Neural_network import DifferentialNeuralNetwork
from decryption import decrypt_image

In [56]:
PASSWORD = "ThisIsEncryptionPassword"

In [57]:
input_image = Image.open("images.jpg").convert("L")

In [58]:
image_array = np.array(input_image)


In [59]:
key = create_sha_key(PASSWORD)
[x,r] = calculate_r_and_x(PASSWORD)

Image Encryption

substitution


In [60]:
T = []
for b in image_array:
    substitute_block = Substitute(b)
    T.append(substitute_block)
T = np.array(T)

perbutation


In [61]:
[c_perbutation , r_perbutation] = calculate_r_and_x(PASSWORD)

In [62]:
perbutated_image =Perturbation(T , r_perbutation ,c_perbutation)

In [63]:
perbutated_image

array([[ 79, 223,  95, ..., 139, 252, 117],
       [153, 124, 221, ...,  99, 150,  97],
       [ 41, 121,  18, ...,  73, 175,  32],
       ...,
       [ 90, 170,  75, ..., 136, 159,  44],
       [205,   0, 123, ..., 121,  90,  76],
       [200,   8,  50, ...,  46, 153,   9]], shape=(183, 276), dtype=uint8)

Second Substitute

In [64]:
V = []
for p in perbutated_image:
    sec_sub= Substitute(p)
    V.append(sec_sub)
V = np.array(V)

Now we will train Deferentail Neural network

In [65]:
num_neurons = len(PASSWORD)
num_layers = 5 # 1 input + 3 hidden + 1 output
total_weights_needed = (num_layers - 1) * (num_neurons * num_neurons)
W_i = create_weights(x , total_weights_needed)
dnn = DifferentialNeuralNetwork(PASSWORD, W_i, num_neurons=num_neurons)

In [66]:
encrypted_rows = []
for v_i in V:
            # Generate blurring codes for the current row
    codes = dnn.generate_codes_and_update(v_i)
    
    # XOR the row with the codes to get the encrypted row
    encrypted_row = np.bitwise_xor(v_i, codes)
    encrypted_rows.append(encrypted_row)

    # Combine all encrypted rows into the final matrix
C_matrix = np.array(encrypted_rows, dtype=np.uint8)

In [67]:
C_matrix = np.array(C_matrix)

In [68]:
cipher_image  = Image.fromarray(C_matrix)
cipher_image.show()

In [74]:
import random
from forward_pass import Substitute_Inv


In [79]:
B = [random.randint(0, 255) for _ in range(30)]
enc = Substitute(B)

print("Encrypted sample:", enc[:10])
print("Has NaN:", any(isinstance(x, float) and math.isnan(x) for x in enc))
print("Has Inf:", any(isinstance(x, float) and not math.isfinite(x) for x in enc))


Encrypted sample: [97, 145, 202, 211, 38, 235, 86, 98, 99, 68]
Has NaN: False
Has Inf: False


In [80]:
B = [random.randint(0, 255) for _ in range(30)]
enc = Substitute(B)
dec = Substitute_Inv(enc)
print("Match:", B == dec)


ValueError: math domain error

In [69]:
decrypted_img = decrypt_image(C_matrix,PASSWORD)

--- Decryption Started ---
Step 1: Generating keys from password...
Step 2: Reversing Differential Neural Network...
Step 2 complete. Reconstructed V matrix with shape: (183, 276)
Step 3: Reversing second substitution...
Error during inverse substitution (V->P): math domain error
Please ensure forward_pass.py contains Substitute_Inv.


In [70]:
decrypted_img